#### Note: I did not make changes to the Pull data file

In [1]:
import geopandas as gp
import pandas as pd
import numpy as np
import os
from shapely.geometry import Point

Borough Code:

1 - Manhattan

2 - Bronx

3 - Brooklyn

4 - Queens

5 - Staten Island

In [2]:
#Downloading census data with geo_location
#https://data.cityofnewyork.us/api/geospatial/fxpq-c8ku?method=export&format=GeoJSON
census = gp.read_file('../data/external/census_tracts/2010 Census Tracts.geojson')
census.head()

OSError: no such file or directory: '../data/external/census_tracts/2010 Census Tracts.geojson'

In [ ]:
#Converting census tract data into a numeric value
census['census_tract'] = pd.to_numeric(census['ct_2010'])
census['boro_code'] = pd.to_numeric(census.boro_code)

In [ ]:
census.columns

In [ ]:
#Dropping extraneous data
census.drop([u'boro_ct_2010',    u'boro_name',   u'cdeligibil',
            u'ct_2010',      u'ctlabel', u'ntacode',    u'ntaname',         u'puma'], inplace = True, axis = 1)
census.head()

In [ ]:
#Checking size
census.shape

In [ ]:
#Downloading census tract data with population
pop_by_census = pd.read_csv('../data/external/census_tracts/New_York_City_Population_By_Census_Tracts.csv')
pop_by_census.head()

In [ ]:
#Taking census by 2010 population
pop_by_census = pop_by_census[pop_by_census.Year == 2010]
total_pop = pop_by_census['Population'].sum()
pop_by_census['density'] = pop_by_census['Population']/total_pop
pop_by_census.head()

In [ ]:
pop_by_census.drop([u'Borough', u'Year', u'FIPS County Code'], axis = 1, inplace = True)
pop_by_census.head()

In [ ]:
#Renaming columns
pop_by_census.rename(columns = { 'DCP Borough Code' : 'boro_code', 'Census Tract': 'census_tract' }, inplace= True)
# pop_by_census.head()
pop_by_census.shape

In [ ]:
#Merging population with geo_file
census_pop = pd.merge(pop_by_census, census, on = ['census_tract', 'boro_code'], how = 'inner')
census_pop.crs = {'init' :'epsg:4326'}
census_pop.head()

In [ ]:
# Importing CitiBike Data
stations = pd.read_csv('../data/external/citibike_data/citi_bike_station_info.csv')
stations = stations.dropna()
stations.head()

In [ ]:
# ### Point sjoin
# geometry = gp.GeoSeries([Point(xy) for xy in zip(stations.Longitude, stations.Latitude)])
# point_stations = gp.GeoDataFrame(stations, geometry=geometry)
# point_stations.crs = {'init' :'epsg:4326'}
# point_stations.to_file('geo_stations')
# point_citibike_popdensity = gp.sjoin(point_stations, census_pop, how = 'inner', op = 'intersects')
# point_citibike_popdensity.head()

In [ ]:
#Creating Buffer
geometry = gp.GeoSeries([Point(xy) for xy in zip(stations.Longitude, stations.Latitude)])
geometry = geometry.buffer(.0005)  # Using buffer of 0.0005
geo_stations = gp.GeoDataFrame(stations, geometry=geometry)
geo_stations.crs = {'init' :'epsg:4326'}
geo_stations.to_file('geo_stations')
geo_stations.head()

In [ ]:
citibike_popdensity = gp.sjoin(geo_stations, census_pop, how = 'inner', op = 'intersects')
citibike_popdensity.head()

In [ ]:
#Checking Boroughs where citibike stands are placed
citibike_popdensity.boro_code.unique()

In [ ]:
citibike_popdensity.columns

In [ ]:
#Dropping data
citibike_popdensity.drop([u'Location',     u'Latitude',    u'Longitude', u'index_right' ], axis=1, inplace=True)
citibike_popdensity.head()

In [ ]:
citibike_popdensity.shape

In [ ]:
# Count of citibike stations 
(citibike_popdensity.groupby(['Station_id'])[['density']].count()).head(10)

In [ ]:
#Calculating population density around a citibike stand by grouing the data
#Using mean to calculate the average of citibike stops intersecting 3 buffers
grouped_data = citibike_popdensity.groupby(['Station_id', 'boro_code'])[['density']].mean()
grouped_data.reset_index(inplace=True)
grouped_data.head()

In [ ]:
grouped_data.to_csv('..data/processed/pop_density.csv')

In [ ]:
# #Merging with census tract data (0.0005 Buffer)
# merged_citibike_avgdensity = pd.merge(grouped_data, point_citibike_popdensity, on = 'Station_id', how = 'inner' )
# merged_citibike_avgdensity.head()

In [ ]:
# #Merging with census tract data (Point)
# merged_citibike_avgdensity = pd.merge(grouped_data, citibike_popdensity, on = 'Station_id', how = 'inner' )
# merged_citibike_avgdensity.head()

In [ ]:
# merged_citibike_avgdensity.columns

In [ ]:
# # For buffer = 0.0005
# merged_citibike_avgdensity.drop([ u'Station_Name', u'geometry',  u'boro_code_y',    u'Population', \
#             u'density',   u'shape_area',   u'shape_leng', 'census_tract'], axis = 1, inplace  = True)

In [ ]:
#Dropping unnecessary data
# # For point buffer
# merged_citibike_avgdensity.drop([ u'Station_Name', u'geometry',  u'boro_code_y', u'Population', \
#             u'density',   u'shape_area',   u'shape_leng', u'Latitude',    u'Longitude', \
#                             u'index_right',  u'Location', 'census_tract'], axis = 1, inplace = True)

In [ ]:
# merged_citibike_avgdensity.head()

In [ ]:
# #Average population density per citibike stand 
# merged_citibike_avgdensity.rename(columns={'boro_code_x' : 'Borough_code'}, inplace = True)
# merged_citibike_avgdensity.head()

In [ ]:
# merged_citibike_avgdensity = merged_citibike_avgdensity.groupby(['Station_id']).mean()
# merged_citibike_avgdensity.reset_index(inplace=True)

In [ ]:
# merged_citibike_avgdensity.head()

### Questions:
1. Is census tract data a good reference dataset to calculate population density around a citibike station? Do we need a larger or smaller dataset?
2. Should we capture the census tract information from where the population density is calculated?
3. Is the population density calculated by the formula density = pop_tract/total_population a good indicator of population density?